In [1]:
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
import plotly.offline as py
py.init_notebook_mode()
import plotly.graph_objs as go
from plotly import tools

## Import and format data

In [129]:
df = pd.read_csv('sales.csv')
df.OrderDate = pd.to_datetime(df.OrderDate)
df.columns = ['date', 'store', 'item', 'sales']
df.date = pd.to_datetime(df.date)
df.item = df.item.astype('category')
df.store = df.store.astype('category')
df.dtypes

date     datetime64[ns]
store          category
item           category
sales             int64
dtype: object

Below are some summary statistics on the data. Overall the quantities for individual items at individual stores is quite small. It would be difficult to forecast daily quantities of individual items and individual stores, so we will work towards forecasting weekly item sales at individual stores.

In [3]:
df.describe(include='all')

,date,store,item,sales
count,350796,350796,350796.0,350796.000000
unique,301,181,39.0,NaN
top,2016-11-05 00:00:00,SEGWD7,41795.0,NaN
freq,4110,5353,22282.0,NaN
first,2016-02-04 00:00:00,NaN,NaN,NaN
last,2016-11-30 00:00:00,NaN,NaN,NaN
mean,NaN,NaN,NaN,1.681211
std,NaN,NaN,NaN,1.134632
min,NaN,NaN,NaN,1.000000
25%,NaN,NaN,NaN,1.000000


Let's take a look at a few cuts on the data to see if we can spot any trends. Below are plots of a handful of individual stores' sales. It looks like the answer to the question about the big jump in sales in September is the addition of a good number of stores.

### Total Sales

In [ ]:
df_total = df.groupby(pd.Grouper(freq='W', key='date')).sum().fillna(0).unstack('date', 0)
df_total.index.levels[1]

len(df_total) == len(df_total.index.levels[1])

trace = go.Scatter(
    x = df_total.index.levels[1],
    y = df_total
)

layout = go.Layout(
    title='Total Sales'
)


fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='total-sales')

### New Stores

In [ ]:
store_sales = df.groupby(['store']+[pd.Grouper(freq='W', key='date')]).sum().fillna(0).unstack('date')
stores_with_sales = store_sales['sales'].where(store_sales.sales > 0).count()

stores_with_sales.index

trace = go.Bar(
    x = stores_with_sales.index,
    y = stores_with_sales
)

layout = go.Layout(
    title='No. of Stores with Sales'
)


fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='stores-with-sales')

### New Items

In [ ]:
item_sales = df.groupby(['item']+[pd.Grouper(freq='W', key='date')]).sum().fillna(0).unstack('date')
items_with_sales = item_sales['sales'].where(item_sales.sales > 0).count()

items_with_sales.index

trace = go.Bar(
    x = items_with_sales.index,
    y = items_with_sales
)

layout = go.Layout(
    title='No. of Items with Sales'
)


fig = go.Figure(data=[trace], layout=layout)
py.iplot(fig, filename='items-with-sales')

## Prepare Data

#### Add records with zero sales so there is a record for every period for every store-item combination

We need to make sure that we add records with zero sales for any combination of store, item and period that doesn't appear in the dataset so that our trailing averages are calculated correctly.

In [130]:
import itertools

beg_date = df['date'].min()
end_date = df['date'].max()
dates = pd.DatetimeIndex(start=beg_date, end=end_date, freq='D')

items = df['item'].value_counts().index
stores = df['store'].value_counts().index

all_periods = pd.DataFrame(list(itertools.product(dates, stores, items)), columns=['date', 'store', 'item'])

In [131]:
all_periods['sales'] = 0
all_periods.date = pd.to_datetime(df.date)
all_periods.item = df.item.astype('category')
all_periods.store = df.store.astype('category')
all_periods.dtypes

date     datetime64[ns]
store          category
item           category
sales             int64
dtype: object

In [132]:
incl_periods = df.groupby(['store', 'item', 'date']).sum().fillna(0).reset_index()
incl_periods.dtypes

store          category
item           category
date     datetime64[ns]
sales           float64
dtype: object

In [133]:
df_all = pd.concat([all_periods, df]).groupby(['store', 'item', 'date']).sum().fillna(0).reset_index()
df_all.head()

,store,item,date,sales
0,SEGWD103,41774,2016-02-04,0.0
1,SEGWD103,41774,2016-02-05,0.0
2,SEGWD103,41774,2016-02-06,1.0
3,SEGWD103,41774,2016-02-07,0.0
4,SEGWD103,41774,2016-02-08,1.0


#### Add Columns for Periods

This time we also want to predict sales for order periods of two and three times per week as well as weekly, assuming orders are placed on the same day each week.

For the **two** orders per week periods, we will predict sales from:
* Tuesday through Thursday (days 2 through 4)
* Friday through Sunday (days 5, 6, 0 and 1)

For the **three** orders per week periods, we will predict sales from:
* Monday through Wednesday (days 1 through 3)
* Thursday and Friday (days 4 and 5)
* Saturday and Sunday (days 6 and 1)

In order to aggregate sales over the correct periods we need to add columns to represent the series for each order period. The end of the two orders per week periods are created in the column `freq2_end` and the three orders per week periods are in `freq3_end`.

We are also adding in columns to distinguish between the intra-weekly periods (which have a zero index) in order to allow the model to compensate for differences in sales volumes between intra-weekly periods, which are `freq2_per` and `freq3_per`.

In [147]:
df_f = df_all.copy()

# Assign each record to its respective intra-week group.
#
# These are the intra-week periods that each day of the week belongs to.
freq = list([[0, 0, 1, 1, 1, 0, 0],
            [0, 1, 1, 1, 2, 2, 0]
            ])

# Map the day of the week of each record to its respective intra-week period.
for i, f in enumerate(freq):
    df_f['freq' + str(i + 2) + '_per'] = df_f['date'].dt.weekday.map(pd.Series(f))

# Assign each record to its respective group within each series of intra-week groups.
# Group membership is indicated in a separate column by the end date of the group.
#
#     1. Calculate the numeric day of the week for each date in the range of dates
#        in the data.
#     2. Create a boolean array with an entry for each record indicating whether
#        the date of the record falls on a day of the week on which an intra-week
#        period ends.
#     3. Calculate the cumulative sum of the boolean area for the range of dates, which
#        will then represent the sequential period each date in our range belongs to.
#     4. Index the cumulative sums by the range of dates to create a lookup table.
#     5. Map the 'date' column in our data to the sequence number using the lookup table.
#     6. Group the lookup table by the period, aggregating the date column by max, which
#        represents the end date of each sequential period, to create another lookup table.
#     7. Map the sequence number series we created earlier to the period ending date using
#        the new lookup table and add it to our data frame.
#

# These are the days of the week that new periods begin on for order frequencies of two
# and three times per week.
period_ends = list([[2, 5],
                   [1, 4, 6]
                   ])

# Range of dates in our data.
dates = pd.date_range(start=df_f['date'].min(), end=df_f['date'].max(), freq='D')

# Execute the same process for each of our order frequencies
for i, p in enumerate(period_ends):
    # Steps 1 through 4
    periods = pd.Series(dates_all.weekday).isin(period_ends[i]).cumsum()
    date_lookup = pd.DataFrame({'date': dates, 'period': periods})
    date_lookup.set_index('date', inplace=True)
    
    # Step 5
    seq_col = df_f['date'].map(date_lookup.period)

    # Step 6
    period_lookup = date_lookup.reset_index().groupby('period').max()

    # Step 7
    df_f['freq' + str(i+2) + '_end'] = seq_col.map(period_lookup.date)

df_f.head()

,store,item,date,sales,freq2_per,freq3_per,freq2_end,freq3_end
0,SEGWD103,41774,2016-02-04,0.0,1,1,2016-02-05,2016-02-04
1,SEGWD103,41774,2016-02-05,0.0,1,2,2016-02-05,2016-02-06
2,SEGWD103,41774,2016-02-06,1.0,0,2,2016-02-09,2016-02-06
3,SEGWD103,41774,2016-02-07,0.0,0,0,2016-02-09,2016-02-08
4,SEGWD103,41774,2016-02-08,1.0,0,0,2016-02-09,2016-02-08


That looks like its working right.

#### Calculating trailing averages
Adding in the rolling average sales is more complicated because we want to calculate the average of like intra-week periods. For example, for the two orders per week frequency the first period spans from Friday through Monday. When we do our trailing averages we want to only include trailing Monday through Friday periods and exclude the Tuesday through Thursday periods that would be included in a strictly sequential calculation. 

In [153]:
# This function calculates the trailing average for a given order frequency. It also
# adds a column for cumulative sales which we user later to filter out records
# for which the store and/or item have no prior sales history.
#
def prep_data(orders_per_week):
    # Check to make sure orders per week is in the available range
    if orders_per_week not in [1, 2, 3]:
        print('Orders per week must be between 1 and 3.')
        raise
    
    freq_per = 'freq' + str(orders_per_week) + '_per'
    freq_end = 'freq' + str(orders_per_week) + '_end'
    freq_end_avg = freq_end + '_avg'
    
    f = {'sales': 'sum', freq_per: 'mean'}
    g = ['store', 'item', freq_end]
    
    # Here we filter the data frame for each of the intra-week periods in
    # the specified order frequency and perform the trailing average and
    # calculations on them separately.
    df_final = pd.DataFrame()
    for i, n in enumerate(df_f[freq_per].value_counts().index):
        df_model = df_f[df_f[freq_per] == n].groupby(g).agg(f).fillna(0)
        
        rolling_sum = (df_model
                       .apply(lambda x:x.rolling(window=6).mean())
                       .shift(1)
                      )

        df_model[freq_end_avg] = rolling_sum['sales']
        df_model['cum_sales'] = df_model.groupby(level=[0,1]).cumsum()['sales']
        df_final = df_final.append(df_model.reset_index())
    
    df_final.columns = df_final.columns.droplevel(1)
      
    return df_final.groupby(g).sum()
    
df_final = prep_data(2)
df_final[df_final['freq2_per'] == 0].head()

freq2_per  sales  freq2_end_avg  cum_sales
store    item  freq2_end                                             
SEGWD103 41774 2016-02-09          0    2.0            NaN        2.0
               2016-02-16          0    1.0            NaN        3.0
               2016-02-23          0    0.0            NaN        3.0
               2016-03-01          0    2.0            NaN        5.0
               2016-03-08          0    2.0            NaN        7.0

### Remove Missing Stores and Items

In [702]:
df_final.describe(include='all')

,freq2,sales,freq2_seq_avg,cum_sales
count,614133.000000,614133.000000,614127.000000,614133.000000
mean,0.505747,0.960316,0.960316,17.922857
std,0.499967,2.105463,1.931207,44.686595
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,1.333333,12.000000
max,1.000000,114.000000,67.666667,1144.000000


In [712]:
df_final_masked = df_final[df_final.cum_sales != 0]
df_final_masked.reset_index(inplace=True)
df_final_masked.describe(include='all')

,store,item,freq2_seq,freq2,sales,freq2_seq_avg,cum_sales
count,274453,274453.0,274453.000000,274453.000000,274453.000000,274442.000000,274453.000000
unique,181,39.0,NaN,NaN,NaN,NaN,NaN
top,SEGWD7,41794.0,NaN,NaN,NaN,NaN,NaN
freq,2623,9265.0,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,55.437284,0.500676,2.148863,1.990261,40.105293
std,NaN,NaN,24.817484,0.500000,2.713943,2.298896,59.822580
min,NaN,NaN,0.000000,0.000000,0.000000,0.000000,1.000000
25%,NaN,NaN,36.000000,0.000000,0.000000,0.333333,5.000000
50%,NaN,NaN,64.000000,1.000000,1.000000,1.333333,16.000000
75%,NaN,NaN,76.000000,1.000000,3.000000,2.833333,51.000000


### Encode Store and Item Categories

It looks like that eliminated quite a few observations, over half of them. Now we can encode the store and item variables as binary classifications and then we can use the data to train a model.

In [714]:
stores = pd.get_dummies(df_final_masked['store'])
items = pd.get_dummies(df_final_masked['item'])
df_final_masked = pd.concat([df_final_masked, stores, items], axis=1).dropna(how='any')
df_final_masked.drop(['freq2_seq', 'cum_sales', 'store', 'item', 41793, 'SEGWD103'], axis=1, inplace=True)
# df_final.to_csv('modeldata.csv')

In [715]:
cols = df_final_masked.columns.tolist()
cols = [cols[1]] + cols
del cols[2]
cols
df_final_masked = df_final_masked[cols]
df_final_masked.head()

,sales,freq2,freq2_seq_avg,SEGWD104,SEGWD116,SEGWD12,SEGWD123,SEGWD125,SEGWD129,SEGWD135,...,42043,42044,42045,42046,42047,42048,42049,42050,42051,42052
11,2.0,1,1.666667,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,4.0,0,1.500000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,1.0,1,2.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,5.0,0,1.833333,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1.0,1,2.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [706]:
data = df_final_masked.dropna().as_matrix()
data.shape

(274448, 221)

### Split Train and Test Data

This time we'll train the model with data through the end of October and then use the remaining data to test model against.

In [716]:
np.random.shuffle(data)
split = int(0.70*data.shape[0])
X_train = np.ones(data[:split,:].shape)
X_test = np.ones(data[split:,:].shape)

X_train[:,1:] = data[:split,1:]
y_train = data[:split,0]

X_test[:,1:] = data[split:,1:]
y_test = data[split:,0]

## Train Model

This is where we actually train the model. I ran it for 200 iterations - more won't likely increase the predictive power of the model, but there are some other diagnostics we can run to see what other improvements we can make.

In [717]:
from sklearn import linear_model
clf = linear_model.SGDRegressor(n_iter=200)
clf.fit(X_train, y_train)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=200, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

## Evaluate Model

In [718]:
predict = clf.predict(X_test)
predict_neg = predict < 0
error = y_test - predict
error_neg = predict_neg @ error
np.savetxt('modelparams.csv', clf.predict(np.eye(X_test.shape[1])), delimiter=",")
print('R-squared: {:.{p}f}'.format(clf.score(X_test, y_test), p=4))
print('Total error in sales quantity: {:.{p}f}'.format(sum(error), p=0))
print('Total error as a % of actual: {:.{p}f}%'.format(sum(error) / sum(y_test)*100, p=2))
print('Total error in sales quantity with zero min prediction: {:.{p}f}'.format(error_neg, p=0))
print('Total error as a % of actual with zero min prediction: {:.{p}f}%'.format((sum(error)+error_neg) / sum(y_test)*100, p=2))

R-squared: 0.5995
Total error in sales quantity: 2113
Total error as a % of actual: 1.19%
Total error in sales quantity with zero min prediction: 1474
Total error as a % of actual with zero min prediction: 2.02%


An overall r-square of 85% is pretty good, I think, for really just dealing with three variables - store and item numbers and historical sales - and the overall error is very low on such a large number of test observations.

In [710]:
predicted = go.Bar(
    name = 'predicted',
    y = clf.predict(X_test)
)

actual = go.Bar(
    name = 'actual',
    y = y_test
)

layout = go.Layout(
    title='Actual vs. Predicted'
)

fig = go.Figure(data=[actual, predicted], layout=layout)
py.iplot(fig, filename='actual-vs-predicted')

Overall this says that this is a pretty good model. The total error over 40,000 individual store-item observations is really low. The graph above shows that we are missing on the big spikes, but overall the performance is good.

## Next Steps

1. Further analyze data
2. Run model diagnostics
3. Analyze model errors by hand
4. Evaluate alternative model hyperparameters
5. Complete Azure implementation

In [ ]:
df_final.loc[:100].to_csv('samplemodeldata.csv')

In [ ]:
np.eye(X_test.shape[1]) * clf.predict(X_test)